<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Limpieza de Datos

En este laboratorio usaremos el DataFrame de Netflix completo creado en los primeros laboratorios de Pandas. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Documenta todas las funciones creadas durante el ejercicio. 

4. Debes incluir después de cada gráfica la interpretación de las mismas en una celda de markdown. 

In [1]:
# Antes de empezar importamos las librerías necesarias
# Para tratamiento de datos
import numpy as np
import pandas as pd
# Para generar todas las combinaciones posibles
import itertools
# Para guardar DataFrames en Excel
from pandas import ExcelWriter
# Para gestión de fechas
from datetime import datetime

# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

# Configuración para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_columns', None) 


In [2]:
# Cargamos el archibo pkl generado en laboratorios anteriores
## Cargamos el archivo
df_catalogo = pd.read_pickle("datos/catalogo_completo.pkl")
df_catalogo.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,yes


## Parte 1: Limpieza y Preparación de Datos

#### Ejercicio 1: Estandarización y limpieza de columnas

En este ejercicio, debes limpiar y estandarizar algunas columnas clave para hacerlas más manejables y consistentes en tus análisis. Específicamente, trabajarás con las columnas `date_added` y `duration` para convertirlas a un formato uniforme y estructurado.

Instrucciones:

1. **Convertir la columna `date_added`**: La columna `date_added` contiene fechas en formato de texto. Debes convertirla a un formato `datetime` que pandas pueda entender y manejar fácilmente.

2. **Limpiar la columna `duration`**: La columna `duration` tiene valores en diferentes formatos como "1 Season", "2 Seasons", "90 min", etc. Tu tarea es extraer el número (ya sea el número de temporadas o la cantidad de minutos) y crear una nueva columna llamada `duration_cleaned` con esos valores estandarizados.


**Resultado Esperado:**
Deberás obtener algo como esto:

| duration   | duration_cleaned |
|------------|-----------------|
| 1 Season   | 1               |
| 90 min     | 90              |
| 2 Seasons  | 2               |
| 45 min     | 45              |
| 3 Seasons  | 3               |

In [3]:
# Vamos a comprobar el formato de las columnas de nuestro archivo
df_catalogo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, 0 to 8806
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8807 non-null   object        
 1   type          8807 non-null   object        
 2   title         8807 non-null   object        
 3   director      6173 non-null   object        
 4   cast          7982 non-null   object        
 5   country       7976 non-null   object        
 6   date_added    8797 non-null   datetime64[ns]
 7   release_year  8807 non-null   int64         
 8   rating        8803 non-null   object        
 9   duration      3994 non-null   object        
 10  listed_in     8807 non-null   object        
 11  description   8807 non-null   object        
 12  Genre         513 non-null    object        
 13  Premiere      513 non-null    datetime64[ns]
 14  Runtime       513 non-null    float64       
 15  IMDB Score    513 non-null    float64      

In [4]:
# Podemos ver que la columna date_added ya tiene formato datetime
# El código necesario sería este: df_catalogo["date_added"] = pd.to_datetime(df_catalogo["date_added"], format="mixed")

In [5]:
# Vamos a generar la columna 'duration_cleaned' separando el valor numérico del texto
df_catalogo["duration_cleaned"] = df_catalogo["duration"].str.split(pat=" ", n=1, expand=True)[0]
# Creamos la columna aplicando el método str.plit a la columna "duration" indicando que el separador va a ser el espacio (pat=" ")
# n=1 indica que solo queremos que haga una división
# expand=True para señalar que queremos que separe cada elemento en columnas distintas
# [0] para que solo recoja en la columna nueva el primer elemento de la lista generada, en este caso sería el número
df_catalogo.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,yes,90
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaT,NaN,NaN,NaN,NaN,2


#### Ejercicio 2: Normalización de la columna `rating`

La columna `rating` tiene diferentes calificaciones como `PG`, `PG-13`, `R`, entre otras. Debes categorizar estas calificaciones en tres grupos:

- **'General Audience'** para calificaciones como `G`, `PG`.

- **'Teens'** para calificaciones como `PG-13`, `TV-14`.

- **'Adults'** para calificaciones como `R`, `TV-MA`.


In [6]:
# Vamos a ver los valores únicos de la columna 'rating'
df_catalogo["rating"].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

Las categorías serán las siguientes:
    - 'General Audience' = "PG", "TV-PG", "TV-G", "G", "TV-Y", "TV-Y7", "TV-Y7-FV"
    - 'Teens' = "PG-13", "TV-14"
    - 'Adults' = "TV-MA", "R", "NR", "UR", "NC-17"
Ignoramos los valores nulos "nan" y los valores en minutos porque corresponden a otra columna

In [7]:
#definimos la función que vamos a aplicar
def clasificacion_edades(categoria):
    """clasifica en grupos de edad según la categoría

    Args:
        categoria (str)): categoría de la serie o película a clasificar

    Returns:
        str: _grupo de edad que corresponde a la categoría
    """
    # Generamos un diccionario que contiene las categorías que corresponden a cada grupo de edad
    dict_categoria = {"General Audience": ["PG", "TV-PG", "TV-G", "G", "TV-Y", "TV-Y7", "TV-Y7-FV"], "Teens": ["PG-13", "TV-14"], "Adults": ["TV-MA", "R", "NR", "UR", "NC-17"]}
    # bucle que compruebe si la categoría está en el el diccionario y nos devuelve la clave correspondiente
    for clave, valor in dict_categoria.items():
        if categoria in valor:
            return clave

In [8]:
df_catalogo["rating_age"] = df_catalogo["rating"].apply(clasificacion_edades)
df_catalogo.sample()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age
5328,s5329,Movie,The Outcasts,Peter Hutchings,"Victoria Justice, Eden Sher, Ashley Rickards, ...",United States,2017-08-14,2017,TV-14,NaN,Comedies,After failing to strike a truce with their sch...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Teens


#### Ejercicio 3: Creación de una columna personalizada basada en el elenco

Vamos a identificar si un actor clave como `Leonardo DiCaprio`, `Tom Hanks`, o `Morgan Freeman` aparece en el elenco.

Usa `apply` y una función lambda para crear una nueva columna llamada `has_famous_actor` que contenga `True` si alguno de estos actores está en la lista de `cast` y `False` en caso contrario.

In [9]:
# Creamos lista de actores famosos
actores_famosos =["Leonardo DiCaprio", "Tom Hanks", "Morgan Freeman"]
# aplicamos una lambda sobre la columna "cast" (x) que devuelve True si cualquier actor de x aparece en la lista de actores que hemos definido
df_catalogo["has_famous_actor"] = df_catalogo["cast"].apply(lambda x : True if any(actor in str(x) for actor in actores_famosos) else False)
df_catalogo[["cast", "has_famous_actor"]].sample(5)


,cast,has_famous_actor
732,"Cast members of the ""To All the Boys"" films di...",False
4094,"Roshan, Priyaa Lal, Aadukalam Naren, Singam Pu...",False
2638,"Ahmed Helmy, Laila Ezz El Arab, Mahmoud El Fis...",False
2084,"Auli'i Cravalho, Justina Machado, Rhenzy Feliz...",False
6408,"Sara Maldonado, Erik Hayser, Andrés Palacios, ...",False


#### Ejercicio 4: Creación de una columna personalizada usando lógica condicional

Vamos a crear una columna llamada `is_recent` que identifique si un título fue lanzado en los últimos 5 años.

Crea una función para marcar con `True` si el título es reciente (lanzado en los últimos 5 años) y `False` si no lo es.

In [10]:
df_catalogo["is_recent"] = df_catalogo["release_year"].apply(lambda x : True if x>= (pd.Timestamp.now().year -5) else False)
# creamos una nueva columna aplicando una función lambda sobre la columna "release_year" del dataframe df_catalogo, que será x en la lambda
# si x es mayor o igual al año actual - 5, es decir, en los últimos 5 años, la lambda devuelve True, en otro caso devuelve False
df_catalogo.sample(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent
3284,s3285,TV Show,Westside Story,NaN,"Tony Sun, Wallace Huo, Esther Liu, Cyndi Wang,...",NaN,2019-11-15,2003,TV-14,NaN,"International TV Shows, Romantic TV Shows, TV ...",Spirited high school misfits with hearts of go...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Teens,False,False
5630,s5631,Movie,Take the 10,Chester Tam,"Josh Peck, Tony Revolori, Kevin Corrigan, Ches...",United States,2017-01-20,2017,TV-MA,80 min,Comedies,Two buddies working dead-end cashier jobs endu...,Comedy,2017-01-20,80.0,4.8,English,yes,80,Adults,False,False
6968,s6969,Movie,Highly Strung,Scott Hicks,NaN,Australia,2017-02-01,2015,TV-14,80 min,"Documentaries, International Movies, Music & M...",Fiery passions take center stage in this intim...,NaN,NaT,NaN,NaN,NaN,NaN,80,Teens,False,False


#### Ejercicio 5: Clasificación de películas por década

En este ejercicio, tu objetivo es categorizar los años de lanzamiento de las películas o series en décadas. La columna `release_year` contiene el año de lanzamiento y debes crear una nueva columna llamada `decade` que indique la década correspondiente, como "1990s", "2000s", etc.


In [11]:
# vamos a ver los valores máximo y mínimo de la columna "release_year"
df_catalogo["release_year"].max()
print(f"El valor mínimo de la columna 'release_year' es {df_catalogo["release_year"].min()} y el máximo es {df_catalogo["release_year"].max()}")

El valor mínimo de la columna 'release_year' es 1925 y el máximo es 2021


In [12]:
# usaremos el método pd.cut para agrupar los años en sus décadas
# definimos las listas de bins y labels que luego usaremos en el método
years_bins = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030]
decadas = ["1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]
# marcamos que se incluya el extremo izquierdo de cada intervalo 'right = False'

In [13]:
df_catalogo["decade"] = pd.cut(df_catalogo["release_year"], bins = years_bins, right = False, labels = decadas)
df_catalogo.sample(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade
3680,s3681,TV Show,Free Rein,NaN,"Jaylen Barron, Manpreet Bambra, Celine Buckens...",United States,2019-07-06,2019,TV-G,NaN,"British TV Shows, Kids' TV, TV Dramas",A 15-year-old from LA spends the summer at her...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,General Audience,False,True,2010s
4947,s4948,Movie,A Sort of Family,Diego Lerman,"Bárbara Lennie, Daniel Aráoz, Claudio Tolcachi...","Argentina, Brazil, France, Poland, Germany, De...",2018-04-01,2017,TV-14,NaN,"Dramas, International Movies, Thrillers",An Argentine doctor faces legal and ethical ch...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Teens,False,False,2010s


#### Ejercicio 6: Extracción de información

Para practicar la extracción de información:

1. **Extrae el primer actor** de la lista en la columna `cast` y crea una nueva columna llamada `first_actor`.

2. **Extrae el primer nombre del director** y guárdalo en una columna llamada `first_name_director`.


In [14]:
# En ambos casos usamos el método .str.split. En los dos marcamos que queremos una columna nueva (expand = True) con el primer elemento obtenido poniendo el [0] al final
# Para extraer el primer actor usamos como separador la coma, ya que los nombres del cast está separados por comas
df_catalogo["first_actor"] = df_catalogo["cast"].str.split(pat=",", n=-1, expand=True)[0]
# Para extraer el primer nombre del director usamos el espacio como separador, para que extraiga solo la primera palabra
df_catalogo["first_name_director"] = df_catalogo["director"].str.split(pat=" ", n=-1, expand=True)[0]
df_catalogo.sample(2)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director
2329,s2330,TV Show,All For Love,NaN,"Ana María Estupiñán, Carlos Torres, Yuri Varga...",Colombia,2020-06-26,2020,TV-MA,NaN,"International TV Shows, Romantic TV Shows, Spa...",A penniless country boy goes in search of his ...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,True,2020s,Ana María Estupiñán,NaN
8785,s8786,TV Show,YOM,NaN,"Sairaj, Devyani Dagaonkar, Ketan Singh, Mayur ...",NaN,2018-06-07,2016,TV-Y7,NaN,Kids' TV,"With the mind of a human being, and the body o...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,General Audience,False,False,2010s,Sairaj,NaN


#### Ejercicio 7: Limpieza de la columna `cast`

La columna `cast` contiene una lista de actores separados por comas. Tu objetivo es realizar las siguientes tareas:

1. **Reemplaza los valores nulos** en la columna `cast` por "sin información".

2. **Contar el número de actores** en cada entrada y crear una nueva columna llamada `num_cast`.

3. **Normalizar los nombres**: Asegúrate de que los nombres de los actores estén en un formato consistente (por ejemplo, quitar espacios adicionales).


In [15]:
# Usamos el método fillna() para rellenar los valores nulos con un valor específico, en este caso "sin información"
df_catalogo["cast"] = df_catalogo["cast"].fillna("sin información")
# Aplicamos el método str.strip() para eliminar posibles espacios en blanco
df_catalogo["cast"] = df_catalogo["cast"].str.strip()
# aplicamos una lambda que nos de la longitud de la lista generada al separar los elementos del cast en cada fila
df_catalogo["num_cast"] = df_catalogo["cast"].apply(lambda x: len(x.split(",")))
df_catalogo.sample(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director,num_cast
3960,s3961,Movie,The Burial of Kojo,"Sam ""Blitz"" Bazawule","Cynthia Dankwa, Ama K. Abebrese, Joseph Otsima...","Ghana, United States",2019-03-31,2018,TV-14,81 min,"Dramas, Independent Movies, International Movies",When a man is left to die inside an illegal go...,NaN,NaT,NaN,NaN,NaN,NaN,81,Teens,False,False,2010s,Cynthia Dankwa,Sam,8
1297,s1298,Movie,No Escape Room,Alex Merkin,"Jeni Ross, Mark Ghanimé, Hamza Haq, Kathyrn Da...",United States,2021-02-18,2018,TV-14,85 min,Horror Movies,A lighthearted bonding opportunity takes a dar...,NaN,NaT,NaN,NaN,NaN,NaN,85,Teens,False,False,2010s,Jeni Ross,Alex,6



#### Ejercicio 9: Identificación de Directores Recurrentes

En este ejercicio, debes identificar los directores que aparecen más de una vez en el conjunto de datos. Realiza los siguientes pasos:

1. **Reemplaza los valores nulos** en la columna `director` por "sin información".

3. **Cuenta cuántas veces aparece cada director** en la columna creada en el ejercicio 6.

4. **Filtra aquellos directores que aparecen más de una vez** y crea una nueva columna llamada `recurrent_director` donde se indique "Yes" si el director aparece varias veces o "No" en caso contrario.

In [16]:
# Usamos el método fillna() para rellenar los valores nulos con un valor específico, en este caso "sin información"
df_catalogo["director"] = df_catalogo["director"].fillna("sin información")
df_catalogo.sample(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director,num_cast
4069,s4070,Movie,Beauty and the Bestie,Wenn V. Deramas,"Vice Ganda, Coco Martin, James Reid, Nadine Lu...",Philippines,2019-02-27,2015,TV-14,119 min,"Action & Adventure, Comedies, International Mo...","Finding himself in a desperate situation, unde...",NaN,NaT,NaN,NaN,NaN,NaN,119,Teens,False,False,2010s,Vice Ganda,Wenn,11
3612,s3613,TV Show,Oh My Ghost,sin información,"Park Bo-young, Cho Jung-seok, Lim Ju-hwan, Kim...",South Korea,2019-08-08,2015,TV-MA,NaN,"International TV Shows, Korean TV Shows, Roman...","Possessed by the ghost of a lustful virgin, a ...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False,2010s,Park Bo-young,NaN,4


In [17]:
# Agrupamos por los directores obtenidos en el ejercicio 6 y usamos size para que cuente las apariciones
apariciones_director = df_catalogo.groupby("first_name_director").size().reset_index(name = "apariciones")
# usamos reset_index(name = "apariciones") para que aparezca una segunda columna llamada apariciones en el nuevo dataframe
apariciones_director

,first_name_director,apariciones
0,A.,4
1,A.R.,2
2,Aadish,1
3,Aamir,2
4,Aanand,1
...,...,...
2303,Çagan,1
2304,Ísold,1
2305,Óskar,1
2306,Ömer,2


In [18]:
# filtramos los nombres de directores cuyo número de apariciones es mayor a 1
directores_recurrentes = apariciones_director[apariciones_director["apariciones"] > 1]["first_name_director"]
# convertimos la serie generada a una lista usando el método tolist()
lista_directores_recurrentes = directores_recurrentes.tolist()

In [19]:
# filtramos si los nombres de la columna "first_name_director" están en la lista de directores recurrentes
df_catalogo["recurrent_director"] = df_catalogo["first_name_director"].isin(lista_directores_recurrentes)
# como el resultado es de tipo bool, aplicaremos un map para cambiarlo a "Yes" o "No"
mapa_recurrentes = {True : "Yes", False: "No"}
df_catalogo["recurrent_director"] = df_catalogo["recurrent_director"].map(mapa_recurrentes)
df_catalogo.sample(5)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director,num_cast,recurrent_director
6338,s6339,Movie,Blue Is the Warmest Color,Abdellatif Kechiche,"Léa Seydoux, Adèle Exarchopoulos, Salim Kechio...","France, Belgium, Spain",2016-08-26,2013,NC-17,180 min,"Dramas, Independent Movies, International Movies","Determined to fall in love, 15-year-old Adele ...",NaN,NaT,NaN,NaN,NaN,NaN,180,Adults,False,False,2010s,Léa Seydoux,Abdellatif,13,No
7416,s7417,Movie,Mary and the Witch's Flower,Hiromasa Yonebayashi,"Ruby Barnhill, Kate Winslet, Jim Broadbent, Ew...",Japan,2018-07-17,2017,PG,NaN,"Anime Features, Children & Family Movies",Ordinary girl Mary picks an extraordinary flow...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,General Audience,False,False,2010s,Ruby Barnhill,Hiromasa,9,Yes
6282,s6283,Movie,Beneath the Leaves,Adam Marino,"Mira Sorvino, Kristoffer Polaha, Doug Jones, P...",United States,2019-06-08,2019,TV-MA,90 min,"Independent Movies, Thrillers",An escaped psychopathic child killer faces off...,NaN,NaT,NaN,NaN,NaN,NaN,90,Adults,False,True,2010s,Mira Sorvino,Adam,11,Yes
3231,s3232,Movie,True: Winter Wishes,sin información,"Michela Luci, Jamie Watson, Eric Peterson, Ann...",NaN,2019-11-26,2019,TV-Y,46 min,Children & Family Movies,An ice crystal from a frosty realm is freezing...,NaN,NaT,NaN,NaN,NaN,NaN,46,General Audience,False,True,2010s,Michela Luci,NaN,8,No
931,s932,Movie,Fun with Dick & Jane,Dean Parisot,"Jim Carrey, Téa Leoni, Alec Baldwin, Richard J...",United States,2021-05-01,2005,PG-13,NaN,Comedies,"After losing their high-paying corporate jobs,...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,Teens,False,False,2000s,Jim Carrey,Dean,10,Yes


In [20]:
# guardamos el archivo final
df_catalogo.to_pickle("datos/catalogo_completo_final.pkl")